In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
import os
import matplotlib.pyplot as plt

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgbm

In [ ]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, GaussianNoise
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint,Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA # swa


# 데이터 불러오기

In [ ]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [ ]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)

In [ ]:
x=x.astype('float32')
test_x=test_x.astype('float32')

# Train_Test_SpIit

In [ ]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
x_train, x_test, y_train, y_test = train_test_split(x,y, \
                                                    train_size=0.8, shuffle=True ,random_state=42)

# Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x =scaler2.transform(test_x)

# Variables

In [ ]:
import datetime
start=datetime.datetime.now()
print(start)

# GridSearch for RandomForest

GridSearchCV(cv=2, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [10, 12], 'n_estimators': [100, 300]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
params= {
    'n_estimators': [10,30],
    'max_depth': [8,10,12],
}

In [ ]:
rf = RandomForestClassifier()

In [ ]:
grid_cv =GridSearchCV(rf, params, cv=2, n_jobs=8)
grid_cv.fit(x_train,y_train)

In [ ]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

In [ ]:
#params={'n_estimators':100,'criterion':'entropy', 'n_jobs':-1,\
#        'max_depth':14}

In [ ]:
best_params = grid_cv.best_params_

# Predict

In [ ]:
rf = RandomForestClassifier(**best_params, n_jobs=8, verbose=2)

In [ ]:
rf = BaggingClassifier(rf)

In [ ]:
rf.fit(x_train, y_train)

In [ ]:
print("test acc: {:.4f}".format(rf.score(x_test, y_test)))

In [ ]:
pred=rf.predict_proba(test_x)
pre =rf.predict(test_x)

In [ ]:
from keras.utils import to_categorical
print(to_categorical(pre[0],19))

In [ ]:
end=datetime.datetime.now()
print("걸린 시간:", end-start)
now=datetime.datetime.now()
print(now)

# Model Predict